# Data wrangling over excel files [library testing]

---



> ### Requirements

In [ ]:
!pip install polars==0.15.8
!pip install numpy==1.21.6
!pip install pandas==1.3.5
!pip install xlrd==1.2.0

In [ ]:
import polars as pl
import xlrd as xls
import numpy as np
import os

> ### **A prior implementation**: descriptions are included

In [ ]:
file = "/content/data_income/existencias-animales.xlsx"
file = "/content/data_income/actividad-principal-region.xlsx"
file = "/content/data_income/existencias-colmenas.xlsx"

In [ ]:
get_ini = lambda x, xs: [index+1 for (name, pibot, index) in zip(xs, ["><"]+xs, range(len(xs))) if name == pibot]
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
del_duplicate = lambda xs: [name for (name, pibot) in zip(xs, ["><"]+xs) if name != pibot]
## Filling up empty space within the row
filling_data = lambda xs: [ xs[i-1] if ( xs[i]=="" and i>0 ) else xs[i] for i in range(len(xs)) ]

Obtaining indexes for sectoring data content

In [ ]:
work_book = xls.open_workbook(file)
work_book = work_book.sheet_by_index(0)

column = 1

array_col = work_book.col_values(column)[1:]
#array_col = del_duplicate(array_col)

*a,ini,fin = get_indexes("", array_col)

## Filling up empty space within the row
array_col = filling_data(array_col)

## Keeping range that it cotains data
array_col = array_col[ini: fin]
print(array_col)

Selecting columns as arrays
Sizing data of the columns
Data normalization
Storing arrays into fields

This process will finish when there are no more columns with data

In [ ]:
fields = np.array(array_col)

column=2
while True:
  try:
    array_col = work_book.col_values(column)[1:]
    #array_col = del_duplicate(array_col)
    
    ## Filling up empty space within the row
    array_col = filling_data(array_col)

    array_col = array_col[ini:fin]
    column+=1

    ## Data normalization
    array_col = np.char.lower(array_col)
    array_col = np.char.replace(array_col, " ", "_")

    #print(array_col)
    fields = np.row_stack((fields, array_col))
  except:
    print("Ending column")
    break
print(fields)

Removing rows with lots of null values to get title for the dataframe

In [ ]:
array_to_dataframe = np.transpose(fields)

pos = 0
normal_long = len(array_to_dataframe[pos])

while True:
  titles = array_to_dataframe[pos]
  out_arr = np.char.count(titles, sub ='', start=1, end=1)
  actual_long = np.sum(out_arr)
  if not len(out_arr) == actual_long :
    array_to_dataframe = array_to_dataframe[1:]
  else:
    break

titles = np.char.lower(titles)
titles = np.char.replace(titles, " ", "_")

for i in range(len(titles)):
  titles[i] = titles[i] +"_"+ str(i)

print(titles)

Showing the datafram

In [ ]:
df_polar = pl.from_numpy(data=array_to_dataframe, columns=list(titles),)
pl.from_numpy
df_polar = df_polar[1:]
print(df_polar)

file = file.replace("_income","_outcome")
df_polar.write_csv(file.split('.')[0]+'.csv')


> ### **Class CsvFileItem [version 1]**: Object-oriented programming for Encapsulation and abstraction

In [ ]:
class CsvFileItem:
  
  get_ini = lambda x, xs: [index+1 for (name, pibot, index) in zip(xs, ["><"]+xs, range(len(xs))) if name == pibot == '']
  get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
  del_duplicate = lambda xs: [name for (name, pibot) in zip(xs, ["><"]+xs) if name != pibot]
  ## Filling up empty space within the row
  filling_data = lambda xs: [ xs[i-1] if ( xs[i]=="" and i>0 ) else xs[i] for i in range(len(xs)) ]

  def __init__(self, path_file) -> None:
    self.path_file = path_file
    self.name_file = path_file.split('/')[-1]
    self.column=1
    self.dataframe=1
  
  def get_column_workbook(self, column):
    work_book = xls.open_workbook(self.path_file)
    work_book = work_book.sheet_by_index(0)
    array_col = work_book.col_values(column)[1:]

    return array_col

  def get_action_margin(self):
    array_col = self.get_column_workbook(1)
    *a,ini,fin = CsvFileItem.get_indexes("", array_col)
    return ini, fin

  def filling_empty_cells(self, array_col):
    ## Filling up empty space within the row
    array_col = CsvFileItem.filling_data(array_col)
    return array_col

  def data_normalization(self, array_col):
    ## Data normalization
    array_col = np.char.lower(array_col)
    array_col = np.char.strip(array_col)
    array_col = np.char.replace(array_col, " ", "_")
    array_col = np.char.replace(array_col, ".", "")
    array_col = np.char.replace(array_col, ",", "")
    return array_col
  

  def get_columns_with_data(self):
    ini, fin = self.get_action_margin()
    array_col = self.get_column_workbook(self.column)
    array_col = self.filling_empty_cells(array_col)
    array_col = self.filling_empty_cells(array_col)
    array_col = self.filling_empty_cells(array_col)
    array_col = array_col[ini:fin]


    column_collection = np.array(array_col)

    self.column+=1

    while True:
      try:
        array_col = self.get_column_workbook(self.column)
        array_col = self.filling_empty_cells(array_col)
        array_col = self.filling_empty_cells(array_col)
        array_col = self.filling_empty_cells(array_col)
        array_col = array_col[ini:fin]

        self.column+=1

        array_col = self.data_normalization(array_col)

        column_collection = np.row_stack((column_collection, array_col))
      except:
        # "Ending columns"
        break
    return column_collection
 
  def create_folder(self):
    path_outcome = self.path_file.split("/")[:-2] + ['data_outcome/']
    path_outcome = "/".join(path_outcome)
    
    # Check whether the specified path exists or not
    isExist = os.path.exists(path_outcome)
    if not isExist:
      # Create a new directory because it does not exist
      os.makedirs(path_outcome)
    
    return path_outcome

 
  def get_titles_dataframe(self):
    array_to_dataframe = np.transpose(self.get_columns_with_data())
    ## Going over rows
    titles=[]
    pos = 0
    normal_long = len(array_to_dataframe[pos])

    while True:
      titles = array_to_dataframe[pos]
      out_arr = np.char.count(titles, sub ='', start=1, end=1)
      actual_long = np.sum(out_arr)
      out_arr = out_arr.tolist()
      if not len(out_arr) == actual_long :
        array_to_dataframe = array_to_dataframe[1:]
      else:
        break

    #titles = self.data_normalization(titles)
    titles = titles.tolist()

    #if it would have many occurences, just in time 
    if len(titles) >0 :
      titles = [ name+"_"+str(i) for (name,i) in zip(titles, range(len(titles)))  ]

    return titles, array_to_dataframe


  def save_dataframe_csv(self):
    titles, array_to_dataframe = self.get_titles_dataframe()

    df_polar = pl.from_numpy(data=array_to_dataframe, columns=list(titles),)
    
    df_polar = df_polar[1:]
    self.dataframe = df_polar

    path_outcome = self.create_folder()
    path_outcome = path_outcome + self.name_file.split('.')[0]+'.csv'
    print(path_outcome)

    df_polar.write_csv(path_outcome)


# Data wragling over INE's files.

---



### **Class FIleReader** : In order to provide data to Array_xl class

In [ ]:
import os
class FileReader:
  def __init__(self, path):
    self.path = path
    self.list_files = []
  
  def collect_files(self):
    # Get the list of all files and directories
    self.list_files = os.listdir(self.path)
    self.list_files = [ self.path +'/'+ x for x in self.list_files ]

  def show_files(self):
    for x in range(len(self.list_files)):
      print(x+1, "  ", self.list_files[x])

### **Class Array_xl** : data wrangling over INE's files.

In [ ]:
import polars as pl
import xlrd as xls
import numpy as np
import os

class Array_xl:
  work_book = ""
  xl_array = np.array([])
  url_bookincome = ""
  url_bookoutside = ""

  def __init__(self, url_bookincome, url_bookoutside, index=0):
    self.url_bookincome = url_bookincome
    self.url_bookoutside = url_bookoutside
    self.work_book = xls.open_workbook(url_bookincome)
    self.work_book = self.work_book.sheet_by_index(index)
    self.to_ndimention_xls()
    self.create_folder_outcome()

  def to_ndimention_xls(self):
    """ Proccessing data and giving Numpy array format: source excel file """

    rack_book = np.array(['']*self.work_book.ncols, dtype='<U32')
    for i in range(self.work_book.nrows):
      rack_book = np.vstack( ( rack_book, np.array(self.work_book.row_values(i), dtype='<U32') ) )    
    self.xl_array = rack_book
  
  def show_work_book(self):
    """ show work_rows' rows """
    for i in work_book.get_rows():
      print(i)
  
  def show_array_excel(self):
    """ show array excel's rows """
    if len(self.xl_array):
      for i in self.xl_array:
        print(i)
  
  def get_ncols(self):
    _, b = self.xl_array.shape
    return b
  def get_nrows(self):
    a, _ = self.xl_array.shape
    return a

  def extending_values_blanckspaces(self):
    """ extextending values toward blank cells: source xl_array"""
    nrows, ncols = self.get_nrows(), self.get_ncols()
    rack_rows = np.array([False]*ncols)
    for i in range(nrows):
      array_row = self.xl_array[i]
      for index in range(len(array_row)):##going by columns
        if array_row[index] == '' and index > 0:
          self.xl_array[i, index] = self.xl_array[i, index-1]

  def remove_empty_rows(self):
    """ Removing empty rows: source xl_array """
    nrows, ncols = self.get_nrows(), self.get_ncols()
    c = 0
    while c < nrows:
      nempty = self.xl_array[c].tolist().count('')
      if nempty == ncols:
        self.xl_array = np.delete( self.xl_array, c, 0)
        nrows = self.get_nrows()
      else:
        c+=1

  def remove_empty_cols(self):
    """ removing empty columns: source xl_array"""
    nrows, ncols = self.get_nrows(), self.get_ncols()
    
    for i in range(1, ncols):
      nempty = self.xl_array[:, i-1:i]
      nempty = np.transpose(nempty).tolist()[0].count('')
      if float(nempty/nrows) >= 0.75 :
        self.xl_array = self.xl_array[:, i:]
  
  def is_empty_col(self, col:int):
    """ Verifying if this column is empty: source xl_array"""
    nrows, ncols = self.get_nrows(), self.get_ncols()
    if col < ncols:
      col+=1
      nempty = self.xl_array[:, col-1:col]
      nempty = np.transpose(nempty).tolist()[0].count('')
      if nempty == nrows :
        return True
      else:
        return False
    else:
      print("Col out of range")

  def is_empty_row(self,row:int):
    """## Verifying if this row is empty: source self.xl_array"""
    nrows, ncols = self.get_nrows(), self.get_ncols()
    if row < nrows:
      nempty = self.xl_array[row].tolist().count('')
      if nempty == ncols:
        return True
      else:
        return False
    else:
      print("Row out of range")
    


  def remove_head_comments(self):
    """ Removing comments head 1. By a row with contente between before and after blank line 2 and if it would be the first and unique column with data: source self.xl_array """
    nrows, ncols = self.get_nrows(), self.get_ncols()
    empty = False
    row_remove=[]
    index=0
    while index < int(nrows*0.3):
      if self.is_empty_row(index):
        row_remove.append(index)
      index+=1

    if len(row_remove):
      row_remove = np.arange(row_remove[0],row_remove[-1])
      self.xl_array = np.delete(self.xl_array, row_remove, axis=0)


  def extending_values_blanckspaces_col(self):
    """ Filling blank spaces in columns"""
    self.xl_array = np.transpose(self.xl_array)
    self.extending_values_blanckspaces()
    self.xl_array = np.transpose(self.xl_array)

  def remove_bottom_comments(self):
    """ Removing comments bottom 2. 
    By couting from last row to up and verifying 
    if it would be the first and unique column with data: source self.xl_array """
    nrows, ncols = self.get_nrows(), self.get_ncols()
    empty = False
    row_remove=[]
    for i in range(1, int(nrows*0.3)): 
      # raging up to 30% to the array seeking for comment section
      if not self.is_empty_row(-i):
        row_remove.append(-i)
      else:
        row_remove.append(-i)
        empty = True
        break
    if len(row_remove) and empty:
      self.xl_array = np.delete( self.xl_array, row_remove, axis = 0)
    else:
      print("There is no comment section apparently >", row_remove)


  def create_folder_outcome(self):
    """ Check whether the specified path exists or not. Create a new directory because it does not exist"""
    isExist = os.path.exists(self.url_bookoutside)
    if not isExist:
      os.makedirs(self.url_bookoutside)
  
  def get_titles_dataframe(self):
    nrows, ncols = self.get_nrows(), self.get_ncols()
    #print(list(map(chr, range(65, 91))))
    titles = np.array([i for i in range(ncols)] ).astype(str)
    return titles


  def default_dataframe_csv(self):
    """ Establishing a correlation of integer numbers as titles for dataframe """
    titles = self.get_titles_dataframe()

    df_polar = pl.from_numpy(data=self.xl_array, columns=titles.tolist(),)
    self.dataframe = df_polar
    print(df_polar)

    df_polar.write_csv(self.url_bookoutside+"/"+self.url_bookincome.split('/')[-1].split('.')[0]+'.csv')
  
  def there_are_repeated(self, titles:list):
    ini = len(titles)
    fin = len( list(set(titles)) )
    if ini == fin:
      return False
    else:
      return True

  def asign_correlative_row(self, titles:list):
    for i in range(len(titles)):
      titles[i] = titles[i]+"_"+str(i)
    return titles

  def custom_dataframe_csv(self, row=None, titles=None):
    """ row:int or titles:list Establishing a certain row within self.xl_array as titles for dataframe """
    if titles == None or row == None:
      if row != None:
        row = int(row)
        titles = self.xl_array[row].astype(str).tolist()
        data = self.xl_array[row+1:]
      else:
        data = self.xl_array
        titles = np.array(titles).astype(str).tolist()
        
      if self.there_are_repeated(titles):
        titles = self.asign_correlative_row(titles)
      
      df_polar = pl.from_numpy(data=data, columns=titles)
      self.dataframe = df_polar
      print(df_polar)
      df_polar.write_csv(self.url_bookoutside+"/"+self.url_bookincome.split('/')[-1].split('.')[0]+'.csv')
    else:
      print("Passed arguments is a must or both are not valid at the same time")

  def data_wrangling(self):
    self.remove_head_comments()
    self.remove_bottom_comments()

    self.remove_empty_cols()
    self.remove_empty_rows()

    self.extending_values_blanckspaces()
    self.extending_values_blanckspaces_col()
  
  def data_normalization(self):
    """It will swap: uppercase to lowercase, 
    blanckspace to underscore, and other"""
    self.xl_array = np.char.lower(self.xl_array)
    self.xl_array = np.char.strip(self.xl_array)
    self.xl_array = np.char.replace(self.xl_array, "-", "")
    self.xl_array = np.char.replace(self.xl_array, "  ", " ")
    self.xl_array = np.char.replace(self.xl_array, " ", "_")
    ##self.xl_array = np.char.replace(self.xl_array, ".", "")
    ##self.xl_array = np.char.replace(self.xl_array, ",", "")
  


### **Main class**

In [ ]:
files = FileReader('/content/data_income')
files.collect_files()
files.show_files()

In [ ]:
for url_file in files.list_files:
  print( url_file)
  array_xl = Array_xl(url_file, '/content/data_outcome_ine_cl')

  array_xl.data_wrangling()
  array_xl.data_normalization()
  array_xl.default_dataframe_csv()
  #array_xl.custom_dataframe_csv(1)
  print(array_xl.xl_array.shape)
  

In [ ]:
print( files.list_files[10])
url_file = files.list_files[10]
array_xl = Array_xl(url_file, '/content/data_outcome_ine_cl')

print(array_xl.xl_array.shape)
array_xl.data_wrangling()
print(array_xl.xl_array.shape)

array_xl.default_dataframe_csv()
array_xl.data_normalization()
array_xl.custom_dataframe_csv(1)





print( files.list_files[11])
url_file = files.list_files[11]
array_xl = Array_xl(url_file, '/content/data_outcome_ine_cl')

print(array_xl.xl_array.shape)
array_xl.data_wrangling()
print(array_xl.xl_array.shape)

array_xl.default_dataframe_csv()
array_xl.data_normalization()
array_xl.custom_dataframe_csv(1)





print( files.list_files[12])
url_file = files.list_files[12]
array_xl = Array_xl(url_file, '/content/data_outcome_ine_cl')

print(array_xl.xl_array.shape)
array_xl.data_wrangling()
print(array_xl.xl_array.shape)

array_xl.default_dataframe_csv()
array_xl.data_normalization()
array_xl.custom_dataframe_csv(1)